In [2]:

# notebooks/ensemble_eval.py
import os, sys
from pathlib import Path
import numpy as np
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:.4f}')

# --- Pfad-Setup --------------------------------------------------------------
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- Imports -----------------------------------------------------------------
from src.config import GlobalConfig, OUTPUTS
from src.io_timesplits import load_target
from src.evaluation import rmse, mae
from src.models.ensemble import (
    load_level1_data,
    get_pred_cols,
    apply_equal_weight,
    apply_trimmed_mean,   # use_median=True möglich
    apply_bates_granger,
    apply_ewa,
    tune_stacking_weights,
    apply_stacking,
    save_ensemble_predictions,
    rmse_table,
)

# --- Setupwahl ---------------------------------------------------------------
# Welches Setup (Suffix in den Modellordnern)?
# Erwartete Ordnerstruktur der Basismodelle:
#   outputs/stageA/<model>_setup_I/...
#   outputs/stageB/<model>_setup_I/...
MODEL_SETUP = "I"  # "I" | "II" | "III"

# --- Konfiguration -----------------------------------------------------------
# Basismodelle (Präfix vor '_setup_<X>')
MODEL_NAMES = [
    #"extra_trees",
    #"svr",
    "sfm",
     "lgbm",
    # "tabpfn",
]

# Daraus die konkreten "Tags" für dieses Setup bauen
MODEL_TAGS = [f"{name}_setup_{MODEL_SETUP}" for name in MODEL_NAMES]

BG_WINDOW = 24
EWA_ETA   = 0.1
EWA_DELTA = 1.0
STACKING_RIDGE_LAMBDA = 0.1

print(f"[INFO] Modell-Setup: {MODEL_SETUP}")
print(f"[INFO] Modell-Tags: {MODEL_TAGS}")
print(f"[INFO] OUTPUTS-Basis: {OUTPUTS}")

# --- 1) Zielvariable laden ---------------------------------------------------
cfg = GlobalConfig(preset="thesis")
y_full = load_target()

# --- 2) Level-1 Daten laden: Stage A (für Stacking-Fit) ---------------------
print("[INFO] Lade Stage-A Level-1 Daten (echte OOS für Stacking-Fit) ...")
df_l1_A = load_level1_data(
    model_tags=MODEL_TAGS,
    base_dir=OUTPUTS,
    y_true_series=y_full,
    stage="A",                     # <- Stage-Schalter: Stage A
)
pred_cols_A = get_pred_cols(df_l1_A)
if len(pred_cols_A) < 2:
    raise RuntimeError(
        f"Zu wenige Modelle in Stage A gefunden (gefunden: {pred_cols_A}). Mindestens 2 benötigt."
    )
print(df_l1_A.head())

# --- 3) Stacking-Gewichte auf Stage A fitten & einfrieren -------------------
print("[INFO] Tune Stacking-Gewichte auf Stage-A OOS ...")
stack_weights = tune_stacking_weights(
    df_tune=df_l1_A,
    pred_cols=pred_cols_A,
    ridge_lambda=STACKING_RIDGE_LAMBDA,
)
print("\n--- Gefrorene Stacking-Gewichte ---")
print(pd.Series(stack_weights, index=pred_cols_A).sort_values(ascending=False))
print(f"Summe: {stack_weights.sum():.6f}")

# --- 4) Stage B laden (für Anwendung/Evaluierung) ---------------------------
print("[INFO] Lade Stage-B Level-1 Daten (Anwendung/Evaluierung) ...")
df_l1_B = load_level1_data(
    model_tags=MODEL_TAGS,
    base_dir=OUTPUTS,
    y_true_series=y_full,
    stage="B",                     # <- Stage-Schalter: Stage B
)
pred_cols_B = get_pred_cols(df_l1_B)
print(df_l1_B.head())

# --- 5) Ensembles berechnen (auf vollem Stage-B-Stream) ---------------------
res = df_l1_B.copy()

# Statisch / Frozen
res["ENS_EqualWeight"] = apply_equal_weight(res, pred_cols_B)
res["ENS_Median"]      = apply_trimmed_mean(res, pred_cols_B, use_median=True)

# Stacking: ACHTUNG -> Spaltenreihenfolge zwischen Stage A & B angleichen
# (falls Modelle in A/B-Join unterschiedlich)
order = [c for c in pred_cols_A if c in pred_cols_B]
if len(order) != len(pred_cols_A):
    # Falls Stage B weniger Spalten hat, normalisiere die Gewichte entsprechend
    idxs = [i for i, c in enumerate(pred_cols_A) if c in order]
    w = stack_weights[idxs]
    w = w / (w.sum() + 1e-12)
    res["ENS_Stacking"] = apply_stacking(res, order, w)
else:
    res["ENS_Stacking"] = apply_stacking(res, pred_cols_A, stack_weights)

# Dynamisch (BG/EWA) – auf dem gesamten Stage-B-Stream (keine Kalstart-Bias)
res["ENS_BG_24M"]     = apply_bates_granger(df_l1_B, pred_cols_B, window=BG_WINDOW)
res["ENS_EWA_eta0.1"] = apply_ewa(df_l1_B, pred_cols_B, eta=EWA_ETA, delta=EWA_DELTA)

# --- 6) RMSE-Tabelle ---------------------------------------------------------
all_cols = pred_cols_B + [c for c in res.columns if c.startswith("ENS_")]
rmse_dict = {col: rmse(res["y_true"], res[col]) for col in all_cols}
df_final_rmse = (
    pd.Series(rmse_dict)
    .sort_values()
    .to_frame("RMSE")
    .rename_axis("Model")
)

print("\n--- Finale RMSE (Stage B, kompletter OOS-Stream) ---")
print(df_final_rmse)

# --- 7) (Optional) Speichern -------------------------------------------------
SAVE_ENSEMBLE = False
RUN_NAME = f"ensemble_setup_{MODEL_SETUP}"

if SAVE_ENSEMBLE:
    # Basispfad für Ensemble-Outputs (separat von Basismodell-Läufen)
    ENSEMBLE_BASE_DIR = OUTPUTS / "stageB" / "ensemble"
    ens_series = {col: res[col] for col in res.columns if col.startswith("ENS_")}

    # Zeitreihen speichern (pro Ensemble + ALL)
    save_ensemble_predictions(
        y_true=res["y_true"],
        ens_preds=ens_series,
        save_dir=ENSEMBLE_BASE_DIR,
        run_name=RUN_NAME,
    )

    # RMSE-Übersicht dazu
    out_dir = ENSEMBLE_BASE_DIR / RUN_NAME
    out_dir.mkdir(parents=True, exist_ok=True)
    df_final_rmse.to_csv(out_dir / "ensemble_rmse_summary.csv")
    print(f"[INFO] Ensemble-Outputs gespeichert unter: {out_dir}")



PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
[INFO] Modell-Setup: I
[INFO] Modell-Tags: ['sfm_setup_I', 'lgbm_setup_I']
[INFO] OUTPUTS-Basis: /Users/jonasschernich/Documents/Masterarbeit/Code/outputs
[INFO] Lade Stage-A Level-1 Daten (echte OOS für Stacking-Fit) ...
            y_true  y_pred_sfm_setup_I  y_pred_lgbm_setup_I
1995-03-01 -1.4157              0.4491              -0.1713
1995-04-01  0.5222              0.5412              -0.1713
1995-05-01  0.6494              0.5412              -0.1713
1995-06-01 -1.1613              0.5412              -0.1713
1995-07-01 -0.2611              0.5412              -0.1713
[INFO] Tune Stacking-Gewichte auf Stage-A OOS ...

--- Gefrorene Stacking-Gewichte ---
y_pred_lgbm_setup_I   0.6663
y_pred_sfm_setup_I    0.3337
dtype: float64
Summe: 1.000000
[INFO] Lade Stage-B Level-1 Daten (Anwendung/Evaluierung) ...
            y_true  y_pred_sfm_setup_I  y_pred_lgbm_setup_I
1997-04-01 -0.3876              0.6812              -0.